### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP на implicit данных

Мягкий дедлайн 13 Октября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 20 Октября (Итоговая проверка)

In [ ]:
!pwd

In [2]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp
import sys
import scipy

from resources import DATA_PATH


В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [3]:
ratings = pd.read_csv(f'{DATA_PATH.parent}/ratings.dat', delimiter='::', header=None, 
                      names=['user_id', 'movie_id', 'rating', 'timestamp'], 
                      usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [7]:
movie_info = pd.read_csv(f'{DATA_PATH.parent}/movies.dat', delimiter='::', header=None, 
                         names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [91]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [92]:
users = ratings["user_id"]
movies = ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

In [93]:
user_item_csr.toarray().shape

(6041, 3953)

Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [59]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [60]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [68]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

In [69]:
user_item_csr.toarray().shape

(6041, 3953)

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [14]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [15]:
model.fit(user_item_t_csr)

  0%|          | 0/100 [00:00<?, ?it/s]

[0.016684524851390655,
 0.014646381557699512,
 0.014066072438858672,
 0.013825753077792373,
 0.013713028146341318,
 0.013652835775061978,
 0.013617273582470005,
 0.0135943240155502,
 0.013578446771416794,
 0.01356682859391075,
 0.013557941085235453,
 0.013550904444853226,
 0.01354517092626943,
 0.013540392866336027,
 0.013536332497395442,
 0.013532825328081163,
 0.013529754153729057,
 0.01352703362428756,
 0.013524598067220332,
 0.01352239963727141,
 0.013520399170666646,
 0.013518566515873227,
 0.013516876776163636,
 0.013515310097951937,
 0.013513851345667693,
 0.013512486484841607,
 0.013511204798919794,
 0.013509997199345022,
 0.013508855325735598,
 0.013507773036672811,
 0.013506745065326828,
 0.013505766269127466,
 0.013504833074868803,
 0.013503941640316267,
 0.01350308896229803,
 0.013502272451651616,
 0.013501489501559891,
 0.013500737801123989,
 0.013500015577693923,
 0.01349932114929224,
 0.013498652739303719,
 0.013498008755582581,
 0.013497387670126865,
 0.0134967884236463

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [16]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [17]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [11]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '1838    Mulan (1998)',
 '1526    Hercules (1997)',
 '2618    Tarzan (1999)',
 '2692    Iron Giant, The (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [12]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [13]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [14]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [15]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1182    Aliens (1986)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '2502    Matrix, The (1999)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1892    Rain Man (1988)',
 '3402    Close Encounters of the Third Kind (1977)',
 '1179    Princess Bride, The (1987)',
 '847    Godfather, The (1972)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

## Data Observation

In [23]:
grouped_df = ratings.groupby('user_id').count()

In [44]:
np.repeat(np.arange(4), 4)

array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3])

In [51]:
import scipy.sparse as sp
sp.csr_matrix((np.random.uniform(1, 4, 16), 
              (np.repeat(np.arange(4), 4), np.repeat(np.arange(4), 4)))
             )

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [32]:
print(f'mean and std, N ratings: {np.floor(grouped_df.movie_id.mean())}, {np.floor(grouped_df.movie_id.std())}')

mean and std, N ratings: 165.0, 192.0


In [56]:
np.einsum('ij,ij -> i', np.array([[0, 0], [1, 1]]), np.array([[0, 0], [1, 1]]))

array([0, 2])

In [65]:
np.square(np.linalg.norm(np.array([[1, 1, 1], [3, 3, 3]]), axis=1))

array([ 3., 27.])

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [81]:
!pwd

/Users/diat.lov/GitHub/RecSys-hse-fall-2021/src/hw1


In [8]:
from svd.svd import SVD

In [9]:
svd_model = SVD()
user_item_matrix = svd_model.get_csr_matrix(ratings)

In [23]:
np.random.uniform(1, 0, 5)

array([0.12539938, 0.4320515 , 0.09961664, 0.51228047, 0.21915897])

In [67]:
arr = [1, 2, 3]
b = True
while len(arr) > 3 and b is True:
    arr = [1]
    print('i')

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных